In [119]:
import os
import requests
import cv2
import numpy as np
import urllib.request
import validators
import matplotlib.pyplot as plt
import tritonclient.http as httpclient

def load_image_from_url(url):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open('image.jpg', 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)

        # Load the image from the file
        image = cv2.imread('image.jpg')

        # Remove the temporary image file
        os.remove('image.jpg')

        return image
    else:
        print('Error loading image:', response.status_code)
        return None

In [120]:
image_url = 'https://ultralytics.com/images/bus.jpg'
image = load_image_from_url(image_url)

In [127]:
import cv2

# Open the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Read a frame from the webcam
ret, frame = cap.read()

# Save the frame as an image file
cv2.imwrite('snapshot.png', frame)

# Release the webcam
cap.release()

In [130]:
image = frame

In [132]:
triton_client = httpclient.InferenceServerClient(
    verbose=False,
    url="202.92.159.241:8008"
)

input0_data = image
height, width, channels = input0_data.shape
input0_data = np.expand_dims(image, axis=0)
# Get the shape of the image


inputs = []
inputs.append(httpclient.InferInput("image", [1,height, width, channels], "UINT8"))
inputs[0].set_data_from_numpy(input0_data, binary_data=False)
model_name = "Yolov8x"

results = triton_client.infer(
        model_name,
        inputs)

# Convert the bounding boxes, probabilities, and names to numpy arrays
bboxes = results.as_numpy("bboxes").flatten()
probs = results.as_numpy("probs").flatten()
names = results.as_numpy("names").flatten()

# Convert the bounding boxes and probabilities to float32
bboxes = np.array(bboxes, dtype=np.float32)
probs = np.array(probs, dtype=np.float32)

# Convert the names to a 1D array and decode from UTF-32
names = names.tobytes().decode('utf-32').split("|")
names = names[:-1]

# Assuming `image` is your image
for i in range(0, len(bboxes), 4):
    if probs[i // 4] > 0.5:
        x1, y1, x2, y2 = bboxes[i:i+4]
        # Generate a random color
        color = tuple((np.random.rand(3,) * 255).astype(int).tolist())
        # Draw the bounding box on the image
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        # Draw the prediction name and probability on the image
        cv2.putText(image, f"{names[i // 4]} {probs[i // 4]:.2f}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Display the image
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [134]:
image

array([[[34, 20,  5],
        [34, 20,  5],
        [34, 20,  5],
        ...,
        [ 3,  1,  1],
        [ 3,  1,  1],
        [ 3,  1,  1]],

       [[34, 20,  3],
        [35, 20,  4],
        [34, 20,  5],
        ...,
        [ 2,  0,  0],
        [ 3,  2,  0],
        [ 2,  0,  0]],

       [[34, 20,  5],
        [35, 20,  5],
        [34, 20,  5],
        ...,
        [ 2,  0,  0],
        [ 2,  0,  0],
        [ 2,  0,  0]],

       ...,

       [[ 1,  1,  1],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]]], dtype=uint8)